In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sim.dy.model_intv import ModelIntv
import sim.dy.keys as I
import numpy.random as rd

In [ ]:
n_collect = 200

In [2]:
def collect_vul(pars_post, n_collect=20, coverages=np.linspace(0, 1, 41), tp="VulACF"):
    idx = ['Pop', 'IncR', 'MorR', 
           'ACF_Vul_Footfall', 'ACF_Vul_Screened', 'ACF_Vul_Confirmed', 'ACF_Vul_Yield', 'ACF_Vul_TP',
           'ACF_Vul_DS_Fl', 'ACF_Vul_DR_Fl', 'ACF_Vul_DR_Sl', 
           'ACF_Plain_Footfall', 'ACF_Plain_Screened', 'ACF_Plain_Confirmed', 'ACF_Plain_Yield', 'ACF_Plain_TP', 
           'ACF_Plain_DS_Fl', 'ACF_Plain_DR_Fl', 'ACF_Plain_DR_Sl']
    
    m = ModelIntv()
    
    sel = rd.choice(list(range(len(pars_post))), n_collect)
    sel.sort()

    mss = list()
    mss_stats = list()


    for i in sel:
        pars = pars_post[i]
        y1, pars = m.find_baseline(pars, 2022)

        for k in coverages:    
            _, ms, _ = m.simulate_onward(y1, pars, intv={tp: {'Coverage': k}}, t_end=2031)

            ms = ms[idx].reset_index().assign(Key = i, Coverage=k, p_comorb=pars['p_comorb'], rr=pars['rr_risk_comorb'])
            mss.append(ms)
            
            ms_stats = {'Key': i, 'Coverage': k, 'Pop0': ms.Pop.iloc[0]}

            for key in idx[1:]:
                ms_stats[key] = (ms[key] * ms.Pop).sum()
            
            mss_stats.append(ms_stats)
            
    mss = pd.concat(mss).assign(Type=tp)
    mss_stats = pd.DataFrame(mss_stats).assign(Type=tp)
    return mss, mss_stats

In [3]:
for folder in ['dy_lo', 'dy_hi']:
    print(folder)
    out_path = f'out/{folder}'
    pars_post = json.load(open(f'{out_path}/Post.json', 'r'))
    r_acf = json.load(open(f'{out_path}/R_ACF_Bg.json', 'r'))

    for p in pars_post:
        p.update(r_acf)

    mss0, mss_stats0 = collect_vul(pars_post, n_collect=n_collect, tp='VulACF')
    mss1, mss_stats1 = collect_vul(pars_post, n_collect=n_collect, tp='PlainACF')
    mss = pd.concat([mss0, mss1])
    mss_stats = pd.concat([mss_stats0, mss_stats1])
    mss.to_csv(f'{out_path}/Sim_VulACF_budget.csv')
    mss_stats.to_csv(f'{out_path}/Sim_VulACF_budget_stats.csv')

dy_lo
dy_hi


In [21]:
def collect_fu(pars_post, n_collect=20, coverage=0.05, tp="VulACF"):
    idx = ['Pop', 'IncR', 'MorR', 
           'ACF_Vul_Footfall', 'ACF_Vul_Screened', 'ACF_Vul_Confirmed', 'ACF_Vul_Yield', 'ACF_Vul_TP',
           'ACF_Vul_DS_Fl', 'ACF_Vul_DR_Fl', 'ACF_Vul_DR_Sl', 
           'ACF_Plain_Footfall', 'ACF_Plain_Screened', 'ACF_Plain_Confirmed', 'ACF_Plain_Yield', 'ACF_Plain_TP', 
           'ACF_Plain_DS_Fl', 'ACF_Plain_DR_Fl', 'ACF_Plain_DR_Sl']
    
    intvs = {
        'Baseline': {},
        'Vul_0_0': {'VulACF': {'Coverage': coverage, 'FollowUp': 0, 'Duration': 0}},
        'Vul_2_6': {'VulACF': {'Coverage': coverage, 'FollowUp': 0.5, 'Duration': 2}},
        'Vul_3_3': {'VulACF': {'Coverage': coverage, 'FollowUp': 0.25, 'Duration': 3}},
        'Vul_3_6': {'VulACF': {'Coverage': coverage, 'FollowUp': 0.5, 'Duration': 3}},
    }
    
    
    m = ModelIntv()
    
    sel = rd.choice(list(range(len(pars_post))), n_collect)
    sel.sort()

    mss = list()
    mss_stats = list()


    for i in sel:
        pars = pars_post[i]
        y1, pars = m.find_baseline(pars, 2022)
        
        for key, intv in intvs.items():
            _, ms, _ = m.simulate_onward(y1, pars, intv=intv, t_end=2031)

            ms = ms[idx].reset_index().assign(Key = i, p_comorb=pars['p_comorb'], rr=pars['rr_risk_comorb'])
            mss.append(ms)
            
            ms_stats = {'Key': i, 'Scenario': key, 'Pop0': ms.Pop.iloc[0]}

            for key in idx[1:]:
                ms_stats[key] = (ms[key] * ms.Pop).sum()
            
            mss_stats.append(ms_stats)
            
    mss = pd.concat(mss).assign(Coverage = coverage)
    mss_stats = pd.DataFrame(mss_stats).assign(Coverage = coverage)
    return mss, mss_stats

In [22]:
for folder in ['dy_lo', 'dy_hi']:
    print(folder)
    out_path = f'out/{folder}'
    pars_post = json.load(open(f'{out_path}/Post.json', 'r'))
    r_acf = json.load(open(f'{out_path}/R_ACF_Bg.json', 'r'))

    for p in pars_post:
        p.update(r_acf)
        
    mss, mss_stats = collect_fu(pars_post, n_collect=n_collect, coverage=0.03)
    mss.to_csv(f'{out_path}/Sim_VulACF_followup.csv')
    mss_stats.to_csv(f'{out_path}/Sim_VulACF_followup_stats.csv')

dy_lo
dy_hi


In [23]:
mss_stats

,Key,Scenario,Pop0,IncR,MorR,ACF_Vul_Footfall,ACF_Vul_Screened,ACF_Vul_Confirmed,ACF_Vul_Yield,ACF_Vul_TP,...,ACF_Vul_DR_Sl,ACF_Plain_Footfall,ACF_Plain_Screened,ACF_Plain_Confirmed,ACF_Plain_Yield,ACF_Plain_TP,ACF_Plain_DS_Fl,ACF_Plain_DR_Fl,ACF_Plain_DR_Sl,Coverage
0,9,Baseline,2.383206,0.213136,0.030057,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.03
1,9,Vul_0_0,2.383206,0.202281,0.027561,7.439183,1.322387,0.359560,0.038084,0.011097,...,0.000474,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.03
2,9,Vul_2_6,2.383206,0.194852,0.025619,9.392093,1.322387,0.359644,0.038188,0.011204,...,0.000483,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.03
3,9,Vul_3_3,2.383206,0.190716,0.024461,10.320985,1.322387,0.359456,0.037956,0.010966,...,0.000477,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.03
4,9,Vul_3_6,2.383206,0.193027,0.025104,10.320019,1.322387,0.359646,0.038190,0.011206,...,0.000487,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,1481,Baseline,2.383169,0.192185,0.027763,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.03
496,1481,Vul_0_0,2.383169,0.179972,0.025018,7.434964,1.322367,0.353445,0.017483,0.011791,...,0.000546,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.03
497,1481,Vul_2_6,2.383169,0.172089,0.022944,9.422109,1.322367,0.353521,0.017578,0.011887,...,0.000558,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.03
498,1481,Vul_3_3,2.383169,0.168112,0.021751,10.378909,1.322367,0.353319,0.017327,0.011634,...,0.000551,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.03
